In [56]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [57]:
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

In [67]:
mesh_stern = 'mobley_test/1112_tetrachloroethane/surf_d04_stern'
mesh_diel = 'mobley_test/1112_tetrachloroethane/surf_d04'
atom_data = numpy.loadtxt('mobley_test/1112_tetrachloroethane/radii.xyzr')

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 1.#80.

atom_pos = atom_data[:,:3]
atom_r   = atom_data[:,3]
N_atom = len(atom_r)

eps_well = 0.3598/4.184 # kcal/mol
r_vdw = 1.9080

rho_w = 0.0336 # 1/angs3 number density of water at standard conditions

grid = generate_grid(mesh_diel)

N_panel = grid.leaf_view.entity_count(0)
vertices = grid.leaf_view.vertices 
triangles = grid.leaf_view.elements
elements = list(grid.leaf_view.entity_iterator(0))

area = numpy.zeros(N_panel)
center = numpy.zeros((N_panel,3))
normal = numpy.zeros((N_panel,3))
for i in range(N_panel):
    area[i] = elements[i].geometry.volume
    center[i,:] = numpy.average(elements[i].geometry.corners[:],axis=0)
    v1 = elements[i].geometry.corners[:,1] - elements[i].geometry.corners[:,0]
    v2 = elements[i].geometry.corners[:,2] - elements[i].geometry.corners[:,0]
    normal[i,:] = numpy.cross(v1,v2)/(2*area[i]) 
    

integral_i = numpy.zeros(N_atom)
for i in range(N_atom):
    r_local = center - atom_pos[i,:]
    r_norm = numpy.sqrt(numpy.sum(r_local**2, axis=1))
    rdn = numpy.sum(r_local*normal, axis=1)
    
    A = 4*eps_well*atom_r[i]**12
    B = 4*eps_well*atom_r[i]**6
    
    test= numpy.where(A/(90*r_norm**12)>1e10)[0]
    print test
    print r_norm[test]
    
    integral_i[i] = numpy.sum((-A/(90*r_norm**12) + B/(12*r_norm**6)) * rdn * area)
    
energy = rho_w*numpy.sum(integral_i)
print energy
print integral_i

[]
[]
[]
[]
[115 153 429 443 446 473 811 856 886 903]
[ 0.16091889  0.13523149  0.06086962  0.1507216   0.17221272  0.16409821
  0.14710314  0.13523231  0.11973072  0.111004  ]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
-1.44986539052e+12
[  3.58134463e+02   2.28680351e-01  -4.31507574e+13   4.24777760e-02
   1.89403609e-02   1.13026251e-01   1.04525051e-01   1.68148040e+06]


In [22]:
grid.topology

In [28]:
1.73310520523/0.14104208421

12.287858726261799

In [33]:
print elements[0].geometry